In [94]:
#Import the necessary modules:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re # used to clean the data
#to display the full text on the notebook without truncation
pd.set_option('display.max_colwidth', 150)
!pip install ir_datasets
import ir_datasets
!pip install python-terrier
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [95]:
dataset = ir_datasets.load("beir/arguana")
docs = [doc for doc in dataset.docs_iter()]
docs_df = pd.DataFrame(docs)
docs_df = docs_df.rename(columns={'doc_id': 'docno'})
docs_df = docs_df.rename(columns={'text': 'raw_text'})
queries = [query for query in dataset.queries_iter()]
queries_df = pd.DataFrame(queries)

qrels = [qrel for qrel in dataset.qrels_iter()]
qrels_df = pd.DataFrame(qrels)
qrels_df

,query_id,doc_id,relevance,iteration
0,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02b,1,0
1,test-environment-aeghhgwpe-pro01a,test-environment-aeghhgwpe-pro01b,1,0
2,test-environment-aeghhgwpe-pro03a,test-environment-aeghhgwpe-pro03b,1,0
3,test-environment-aeghhgwpe-pro04a,test-environment-aeghhgwpe-pro04b,1,0
4,test-environment-aeghhgwpe-con02a,test-environment-aeghhgwpe-con02b,1,0
...,...,...,...,...
1401,test-society-epsihbdns-pro04a,test-society-epsihbdns-pro04b,1,0
1402,test-society-epsihbdns-con02a,test-society-epsihbdns-con02b,1,0
1403,test-society-epsihbdns-con04a,test-society-epsihbdns-con04b,1,0
1404,test-society-epsihbdns-con03a,test-society-epsihbdns-con03b,1,0


In [96]:
# data = []
# for i in range(1,101):
#     with open(f"sport_{i}.txt", "r") as file:
#         content = file.read()
#         data.append([f"d{i}", content])
# docs_df = pd.DataFrame(data, columns=["docno", "raw_text"])
# docs_df

In [97]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [98]:
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    # print('Tokens are:',tokens,'\n')
    return ' '.join(filtered_tokens)
docs_df['processed_text'] = docs_df['raw_text'].apply(remove_stopwords)
docs_df

,docno,raw_text,title,processed_text
0,test-environment-aeghhgwpe-pro02b,You don’t have to be vegetarian to be green. Many special environments have been created by livestock farming – for example chalk down land in Eng...,animals environment general health health general weight philosophy ethics,’ vegetarian green . many special environments created livestock farming – example chalk land england mountain pastures many countries . ending li...
1,test-environment-aeghhgwpe-pro02a,Being vegetarian helps the environment Becoming a vegetarian is an environmentally friendly thing to do. Modern farming is one of the main source...,animals environment general health health general weight philosophy ethics,vegetarian helps environment becoming vegetarian environmentally friendly thing . modern farming one main sources pollution rivers . beef farming ...
2,test-environment-aeghhgwpe-pro03b,"The key to good health is a balanced diet, not a meat- and fish-free diet. Meat and fish are good sources of protein, iron, and other vitamins and...",animals environment general health health general weight philosophy ethics,"key good health balanced diet , meat- fish-free diet . meat fish good sources protein , iron , vitamins minerals . health benefits vegetarian diet..."
3,test-environment-aeghhgwpe-pro01a,It is immoral to kill animals As evolved human beings it is our moral duty to inflict as little pain as possible for our survival. So if we do no...,animals environment general health health general weight philosophy ethics,"immoral kill animals evolved human beings moral duty inflict little pain possible survival . need inflict pain animals order survive , . farm anim..."
4,test-environment-aeghhgwpe-pro01b,"There is a great moral difference between humans and animals. Unlike animals, humans are capable of rational thought and can alter the world aroun...",animals environment general health health general weight philosophy ethics,"great moral difference humans animals . unlike animals , humans capable rational thought alter world around . creatures put earth mankind use , in..."
...,...,...,...,...
8669,validation-society-fyhwscdcj-con01b,"Stories about ridiculous administration costs are not only rare but are often untrue. For example, in the case of one of the largest sponsorship o...",,"stories ridiculous administration costs rare often untrue . example , case one largest sponsorship organisations - world vision - “ funds given 20..."
8670,validation-society-fyhwscdcj-con02a,We need to address the causes of poverty rather than treat the symptoms (outward signs). There are better ways to help people. Helping single chi...,,"need address causes poverty rather treat symptoms ( outward signs ) . better ways help people . helping single children , even villages , treats s..."
8671,validation-society-fyhwscdcj-con03a,Sponsorship is often more about the intentions of the donors rather than the needs of poor children. Some schemes have a clear cultural and relig...,,sponsorship often intentions donors rather needs poor children . schemes clear cultural religious motive – desire give aid way affect even impose ...
8672,validation-society-fyhwscdcj-con01a,Sponsorship is an inefficient way of giving to charity. Sponsoring a child is a costly way to do good. More of the money given is taken up with a...,,sponsorship inefficient way giving charity . sponsoring child costly way good . money given taken administration ( organising ) compared ways help...


In [99]:
docs_df

,docno,raw_text,title,processed_text
0,test-environment-aeghhgwpe-pro02b,You don’t have to be vegetarian to be green. Many special environments have been created by livestock farming – for example chalk down land in Eng...,animals environment general health health general weight philosophy ethics,’ vegetarian green . many special environments created livestock farming – example chalk land england mountain pastures many countries . ending li...
1,test-environment-aeghhgwpe-pro02a,Being vegetarian helps the environment Becoming a vegetarian is an environmentally friendly thing to do. Modern farming is one of the main source...,animals environment general health health general weight philosophy ethics,vegetarian helps environment becoming vegetarian environmentally friendly thing . modern farming one main sources pollution rivers . beef farming ...
2,test-environment-aeghhgwpe-pro03b,"The key to good health is a balanced diet, not a meat- and fish-free diet. Meat and fish are good sources of protein, iron, and other vitamins and...",animals environment general health health general weight philosophy ethics,"key good health balanced diet , meat- fish-free diet . meat fish good sources protein , iron , vitamins minerals . health benefits vegetarian diet..."
3,test-environment-aeghhgwpe-pro01a,It is immoral to kill animals As evolved human beings it is our moral duty to inflict as little pain as possible for our survival. So if we do no...,animals environment general health health general weight philosophy ethics,"immoral kill animals evolved human beings moral duty inflict little pain possible survival . need inflict pain animals order survive , . farm anim..."
4,test-environment-aeghhgwpe-pro01b,"There is a great moral difference between humans and animals. Unlike animals, humans are capable of rational thought and can alter the world aroun...",animals environment general health health general weight philosophy ethics,"great moral difference humans animals . unlike animals , humans capable rational thought alter world around . creatures put earth mankind use , in..."
...,...,...,...,...
8669,validation-society-fyhwscdcj-con01b,"Stories about ridiculous administration costs are not only rare but are often untrue. For example, in the case of one of the largest sponsorship o...",,"stories ridiculous administration costs rare often untrue . example , case one largest sponsorship organisations - world vision - “ funds given 20..."
8670,validation-society-fyhwscdcj-con02a,We need to address the causes of poverty rather than treat the symptoms (outward signs). There are better ways to help people. Helping single chi...,,"need address causes poverty rather treat symptoms ( outward signs ) . better ways help people . helping single children , even villages , treats s..."
8671,validation-society-fyhwscdcj-con03a,Sponsorship is often more about the intentions of the donors rather than the needs of poor children. Some schemes have a clear cultural and relig...,,sponsorship often intentions donors rather needs poor children . schemes clear cultural religious motive – desire give aid way affect even impose ...
8672,validation-society-fyhwscdcj-con01a,Sponsorship is an inefficient way of giving to charity. Sponsoring a child is a costly way to do good. More of the money given is taken up with a...,,sponsorship inefficient way giving charity . sponsoring child costly way good . money given taken administration ( organising ) compared ways help...


In [100]:
from nltk.stem import *
from nltk.stem.porter import *

In [101]:
stemmer = PorterStemmer()

In [102]:
def Steem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

In [103]:
docs_df["processed_text"] = docs_df["processed_text"].apply(Steem_text)
docs_df

,docno,raw_text,title,processed_text
0,test-environment-aeghhgwpe-pro02b,You don’t have to be vegetarian to be green. Many special environments have been created by livestock farming – for example chalk down land in Eng...,animals environment general health health general weight philosophy ethics,’ vegetarian green . mani special environ creat livestock farm – exampl chalk land england mountain pastur mani countri . end livestock farm would...
1,test-environment-aeghhgwpe-pro02a,Being vegetarian helps the environment Becoming a vegetarian is an environmentally friendly thing to do. Modern farming is one of the main source...,animals environment general health health general weight philosophy ethics,"vegetarian help environ becom vegetarian environment friendli thing . modern farm one main sourc pollut river . beef farm one main caus deforest ,..."
2,test-environment-aeghhgwpe-pro03b,"The key to good health is a balanced diet, not a meat- and fish-free diet. Meat and fish are good sources of protein, iron, and other vitamins and...",animals environment general health health general weight philosophy ethics,"key good health balanc diet , meat- fish-fre diet . meat fish good sourc protein , iron , vitamin miner . health benefit vegetarian diet deriv hig..."
3,test-environment-aeghhgwpe-pro01a,It is immoral to kill animals As evolved human beings it is our moral duty to inflict as little pain as possible for our survival. So if we do no...,animals environment general health health general weight philosophy ethics,"immor kill anim evolv human be moral duti inflict littl pain possibl surviv . need inflict pain anim order surviv , . farm anim chicken , pig , sh..."
4,test-environment-aeghhgwpe-pro01b,"There is a great moral difference between humans and animals. Unlike animals, humans are capable of rational thought and can alter the world aroun...",animals environment general health health general weight philosophy ethics,"great moral differ human anim . unlik anim , human capabl ration thought alter world around . creatur put earth mankind use , includ eat meat . re..."
...,...,...,...,...
8669,validation-society-fyhwscdcj-con01b,"Stories about ridiculous administration costs are not only rare but are often untrue. For example, in the case of one of the largest sponsorship o...",,"stori ridicul administr cost rare often untru . exampl , case one largest sponsorship organis - world vision - “ fund given 2010 , 81.1 per cent w..."
8670,validation-society-fyhwscdcj-con02a,We need to address the causes of poverty rather than treat the symptoms (outward signs). There are better ways to help people. Helping single chi...,,"need address caus poverti rather treat symptom ( outward sign ) . better way help peopl . help singl children , even villag , treat symptom povert..."
8671,validation-society-fyhwscdcj-con03a,Sponsorship is often more about the intentions of the donors rather than the needs of poor children. Some schemes have a clear cultural and relig...,,sponsorship often intent donor rather need poor children . scheme clear cultur religi motiv – desir give aid way affect even impos ( forc ) foreig...
8672,validation-society-fyhwscdcj-con01a,Sponsorship is an inefficient way of giving to charity. Sponsoring a child is a costly way to do good. More of the money given is taken up with a...,,"sponsorship ineffici way give chariti . sponsor child costli way good . money given taken administr ( organis ) compar way help poor peopl , altho..."


In [104]:
indexer = pt.DFIndexer("./myFirstIndex", overwrite=True)
index_ref = indexer.index(docs_df["processed_text"], docs_df["docno"])

20:08:48.651 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (training-society-iasihbmubf-pro06a) - further warnings are suppressed
20:08:50.857 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


In [105]:
print(index_ref.toString())
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

./myFirstIndex/data.properties
Number of documents: 8674
Number of terms: 22473
Number of postings: 559226
Number of fields: 0
Number of tokens: 750403
Field names: []
Positions:   false



In [106]:
def clean(text):
   text = re.sub(r"http\S+", " ", text) # remove urls
   text = re.sub(r"RT ", " ", text) # remove rt
   text = re.sub(r"@[\w]*", " ", text) # remove handles
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump
   text = re.sub(r"\s+", " ", text) # remove extra white space
   text = text.strip()
   return text
docs_df['processed_text'] = docs_df["processed_text"].apply(clean)
docs_df

,docno,raw_text,title,processed_text
0,test-environment-aeghhgwpe-pro02b,You don’t have to be vegetarian to be green. Many special environments have been created by livestock farming – for example chalk down land in Eng...,animals environment general health health general weight philosophy ethics,’ vegetarian green mani special environ creat livestock farm – exampl chalk land england mountain pastur mani countri end livestock farm would see...
1,test-environment-aeghhgwpe-pro02a,Being vegetarian helps the environment Becoming a vegetarian is an environmentally friendly thing to do. Modern farming is one of the main source...,animals environment general health health general weight philosophy ethics,vegetarian help environ becom vegetarian environment friendli thing modern farm one main sourc pollut river beef farm one main caus deforest long ...
2,test-environment-aeghhgwpe-pro03b,"The key to good health is a balanced diet, not a meat- and fish-free diet. Meat and fish are good sources of protein, iron, and other vitamins and...",animals environment general health health general weight philosophy ethics,key good health balanc diet meat- fish-fre diet meat fish good sourc protein iron vitamin miner health benefit vegetarian diet deriv high fibr low...
3,test-environment-aeghhgwpe-pro01a,It is immoral to kill animals As evolved human beings it is our moral duty to inflict as little pain as possible for our survival. So if we do no...,animals environment general health health general weight philosophy ethics,immor kill anim evolv human be moral duti inflict littl pain possibl surviv need inflict pain anim order surviv farm anim chicken pig sheep cow se...
4,test-environment-aeghhgwpe-pro01b,"There is a great moral difference between humans and animals. Unlike animals, humans are capable of rational thought and can alter the world aroun...",animals environment general health health general weight philosophy ethics,great moral differ human anim unlik anim human capabl ration thought alter world around creatur put earth mankind use includ eat meat reason say m...
...,...,...,...,...
8669,validation-society-fyhwscdcj-con01b,"Stories about ridiculous administration costs are not only rare but are often untrue. For example, in the case of one of the largest sponsorship o...",,stori ridicul administr cost rare often untru exampl case one largest sponsorship organis - world vision - “ fund given 2010 81 1 per cent went di...
8670,validation-society-fyhwscdcj-con02a,We need to address the causes of poverty rather than treat the symptoms (outward signs). There are better ways to help people. Helping single chi...,,need address caus poverti rather treat symptom ( outward sign ) better way help peopl help singl children even villag treat symptom poverti - make...
8671,validation-society-fyhwscdcj-con03a,Sponsorship is often more about the intentions of the donors rather than the needs of poor children. Some schemes have a clear cultural and relig...,,sponsorship often intent donor rather need poor children scheme clear cultur religi motiv – desir give aid way affect even impos ( forc ) foreign ...
8672,validation-society-fyhwscdcj-con01a,Sponsorship is an inefficient way of giving to charity. Sponsoring a child is a costly way to do good. More of the money given is taken up with a...,,sponsorship ineffici way give chariti sponsor child costli way good money given taken administr ( organis ) compar way help poor peopl although co...


In [107]:
def preprocess(sentence):
    sentence = sentence.lower()

    # Remove html tags
    sentence = re.sub(r'<[^>]+>', ' ', sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Tokenize
    words = word_tokenize(sentence)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Join the words back into a single string
    sentence = ' '.join(words)

    return sentence

In [108]:
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)
query="ahmed"
query = preprocess(query)
results=tfidf_retr.search(query)
if len(results)==0:
   print("There are no relevant documents for your selected query.")
else:
   print(results)

  qid  docid                                     docno  rank     score query
0   1   2500          test-religion-msgfhwbamec-pro01a     0  7.566685   ahm
1   1    953       test-international-aglhrilhb-con02b     1  6.179755   ahm
2   1   6557       training-politics-oamepdgtwh-pro02a     2  5.431404   ahm
3   1   5249   training-international-aptwhbfri-pro02a     3  4.883798   ahm
4   1   3018        training-health-ssiahrgmhwc-con02a     4  4.806266   ahm
5   1   4759  training-international-meptwhbi22-con01b     5  4.205402   ahm


In [109]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [110]:
import re
from tqdm import tqdm

pd.set_option('display.max_colwidth', 150)
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-uncased"

bert_tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name)

In [111]:
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git --quiet
!pip install transformers

  Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [112]:
from xpmir.models import AutoModel

model = AutoModel.load_from_hf_hub("xpmir/monot5", as_instance=True)

In [113]:
result_merged = results.merge(docs_df, on="docno")[["score", "processed_text"]]
result_merged = result_merged.sort_values(by="score", ascending=False)
result_merged
output = model.rsv(query, result_merged["processed_text"].values)
data =[(list(obj.document.items.values())[0].text,obj.score) for obj in output]
reviews_result_v2 = pd.DataFrame(data, columns=['document', "score"]).sort_values(by="score", ascending=False)

In [114]:
reviews_result_v2

,document,score
2,progress made egypt biggest arab state one two far larg peac revolut perhap best exampl progress made referendum egypt march amend constitut pass ...,-7.692349
3,fight terror tuareg separatist first spark insurrect want split mali north taken islamist al qaeda islam maghreb present terrorist threat ignor fr...,-9.576002
5,none argument chang fact 250 000 isra southern israel live constant fear hama rocket attack hama escal ceasefir refus extend notabl syria implac e...,-13.345811
4,reduct aid caus innoc peopl suffer reduct aid africa like affect vulner societi rather politician adjust law african contin show sign growth 40 % ...,-13.959131
0,individu freedom even marriag made absolut mandatori covert famili pressur still propel peopl union unhappi form restrict liberti consequ peopl re...,-17.717628
1,prosecut allow equal chanc prosecut defens show truth believ result far fact brought life process reliant individu ‘ truth ’ moreov amnesti may fo...,-18.110519


In [115]:
rm3_expander = pt.rewrite.RM3(index,fb_terms=10, fb_docs=100)
rm3_qe = tfidf_retr >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]

expanded_query

'applypipeline:off 2012^0.030767934 prosecut^0.032143194 israel^0.051337089 terrorist^0.035590678 attack^0.031037502 truth^0.032143194 ahm^0.650686681 egypt^0.043479376 amnesti^0.032143194 hama^0.060671106'

In [116]:
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)
query=expanded_query
results=tfidf_retr.search(query)
if len(results)==0:
   print("There are no relevant documents for your selected query.")
else:
   print(results)

  qid  docid                                     docno  rank     score  \
0   1   2500          test-religion-msgfhwbamec-pro01a     0  7.653169   
1   1    953       test-international-aglhrilhb-con02b     1  7.096738   
2   1   6557       training-politics-oamepdgtwh-pro02a     2  5.868222   
3   1   4759  training-international-meptwhbi22-con01b     3  5.825342   
4   1   5249   training-international-aptwhbfri-pro02a     4  5.379488   
5   1   3018        training-health-ssiahrgmhwc-con02a     5  4.806266   
6   1   4760  training-international-meptwhbi22-con02a     6  1.444028   
7   1   4758  training-international-meptwhbi22-con03b     7  1.432021   
8   1   4763  training-international-meptwhbi22-con02b     8  1.410701   
9   1   4755  training-international-meptwhbi22-pro01a     9  1.365826   

                                                                                                                                                   query  
0  applypipeline:off 2012^0.03

In [117]:
result_merged = results.merge(docs_df, on="docno")[["score", "processed_text"]]
result_merged = result_merged.sort_values(by="score", ascending=False)
result_merged
output = model.rsv(query, result_merged["processed_text"].values)
data =[(list(obj.document.items.values())[0].text,obj.score) for obj in output]
reviews_result_v2 = pd.DataFrame(data, columns=['document', "score"]).sort_values(by="score", ascending=False)

In [118]:
reviews_result_v2

,document,score
6,israel 's militari oper harm chanc peac long term long-term secur israel rest stabl peac agreement palestinian attempt bludgeon hama truce 'on isr...,-0.556114
4,fight terror tuareg separatist first spark insurrect want split mali north taken islamist al qaeda islam maghreb present terrorist threat ignor fr...,-0.881980
3,none argument chang fact 250 000 isra southern israel live constant fear hama rocket attack hama escal ceasefir refus extend notabl syria implac e...,-1.028037
1,prosecut allow equal chanc prosecut defens show truth believ result far fact brought life process reliant individu ‘ truth ’ moreov amnesti may fo...,-2.274058
8,import thing region peac long run belief among isra 'militari solut ' conflict rather belief hama backer syria iran israel 'solv ' militarili beli...,-2.830378
7,indisput hama launch violent attack civilian target israel hand conduct oper exercis due care limit civilian casualti hama terrorist howev set hea...,-4.620934
9,militari oper legitim isra self-defens militari oper legitim use isra state ’ right defend citizen quot then-president-elect barack obama - `` som...,-8.496802
0,individu freedom even marriag made absolut mandatori covert famili pressur still propel peopl union unhappi form restrict liberti consequ peopl re...,-11.017054
5,reduct aid caus innoc peopl suffer reduct aid africa like affect vulner societi rather politician adjust law african contin show sign growth 40 % ...,-13.463023
2,progress made egypt biggest arab state one two far larg peac revolut perhap best exampl progress made referendum egypt march amend constitut pass ...,-14.257401


In [122]:
queries_df

,query_id,query
0,test-environment-aeghhgwpe-pro02a,vegetarian help environ becom vegetarian environment friendli thing modern farm one main sourc pollut river beef farm one main caus deforest long ...
1,test-environment-aeghhgwpe-pro01a,immor kill anim evolv human be moral duti inflict littl pain possibl surviv need inflict pain anim order surviv farm anim chicken pig sheep cow se...
2,test-environment-aeghhgwpe-pro03a,vegetarian healthier signific health benefit go veggi vegetarian diet contain high quantiti fibr vitamin miner low fat vegan diet even better sinc...
3,test-environment-aeghhgwpe-pro04a,vegetarian reduc risk food poison almost danger type food poison pass meat egg campylobact bacteria common caus food poison england usual found ra...
4,test-environment-aeghhgwpe-con02a,problem vegetarian vegetarian vegan diet may result person get enough iron although get iron food puls green leafi veget nut iron food absorb easi...
...,...,...
1401,test-society-epsihbdns-pro04a,poor uneduc peopl lure citi caus rural urban migrat develop nation main reason becom problemat peopl move citi make inform decis led believ citi c...
1402,test-society-epsihbdns-con02a,practic imposs control peopl movement one major problem propos lie fact inde deal develop nation nation limit capac manag kind system happen inste...
1403,test-society-epsihbdns-con04a,restrict caus incred loss potenti one best thing function develop nation young peopl choos profess apart benefici individu mean best suit person g...
1404,test-society-epsihbdns-con03a,rural life miser higher mortal rate citi planet find wors live standard anywher rural area develop countri area famin child mortal diseas aid plag...


In [125]:
# eval = pt.Evaluate(results,qrels_df)
nltk.download('wordnet')
qrels_df = qrels_df.rename(columns={'query_id':'qid'})
qrels_df = qrels_df.rename(columns={'doc_id':'docno'})
queries_df = queries_df.rename(columns={'text':'query'})
queries_df = queries_df.rename(columns={'query_id':'qid'})
queries_df['query'] = queries_df['query'].apply(preprocess)
res = tfidf_retr.transform(queries_df)
eval = pt.Evaluate(res,qrels_df)
eval


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'map': 0.22684611303032304, 'ndcg': 0.34234808901474206}

In [126]:
eval = pt.Evaluate(res,qrels_df,metrics=["map","recall","P"], perquery=True)
eval

defaultdict(dict,
            {'test-environment-aeghhgwpe-pro02a': {'map': 0.5,
              'P@5': 0.2,
              'P@10': 0.1,
              'P@15': 0.06666666666666667,
              'P@20': 0.05,
              'P@30': 0.03333333333333333,
              'P@100': 0.01,
              'P@200': 0.005,
              'P@500': 0.002,
              'P@1000': 0.001,
              'R@5': 1.0,
              'R@10': 1.0,
              'R@15': 1.0,
              'R@20': 1.0,
              'R@30': 1.0,
              'R@100': 1.0,
              'R@200': 1.0,
              'R@500': 1.0,
              'R@1000': 1.0},
             'test-environment-aeghhgwpe-pro01a': {'map': 0.3333333333333333,
              'P@5': 0.2,
              'P@10': 0.1,
              'P@15': 0.06666666666666667,
              'P@20': 0.05,
              'P@30': 0.03333333333333333,
              'P@100': 0.01,
              'P@200': 0.005,
              'P@500': 0.002,
              'P@1000': 0.001,
              'R@5'